# Intro til språkmodeller

## Spørsmål

- Hvordan gjør vi det med nøkler? Per nå leser denne fortsatt fra .env-filen da jeg ikke ville pushe de til github.


# AzureChatOpenAI
### Språkmodellen

In [9]:
from langchain_openai import AzureChatOpenAI
from dotenv import find_dotenv, load_dotenv
import os


# Get environment variables
load_dotenv(find_dotenv(), override=True)


class Llm:
    """
    Class containing the language model.
    """

    llm = AzureChatOpenAI(
        azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_GPT_4O_MINI"],
        model=os.environ.get("OPENAI_MODEL_GPT_4O_MINI", default="gpt-4o-mini"),
        openai_api_key=os.environ["OPENAI_API_KEY_4O"],
        openai_api_version=os.environ["OPENAI_API_VERSION_4O"],
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT_4O"],
        temperature=0,
    )


class LlmRes:
    """
    Class containing the language model.
    """

    llm = AzureChatOpenAI(
        azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_GPT_3O_MINI"],
        model=os.environ.get("OPENAI_MODEL_GPT_3O_MINI", default="o3-mini"),
        openai_api_key=os.environ["OPENAI_API_KEY_3O"],
        openai_api_version=os.environ["OPENAI_API_VERSION_3O"],
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT_3O"],
        reasoning_effort="high",
        # reasoning_effort: str,
        # Constrains effort on reasoning for reasoning models.
        # Reasoning models only, like OpenAI o1 and o3-mini.
        # Currently supported values are low, medium, and high.
        # Reducing reasoning effort can result in faster responses and fewer tokens used on reasoning in a response.
    )


# LLM = Llm()

# print(Llm.llm.invoke("whats up?").content)


# Datasettet

In [10]:
import pandas as pd


class FeedbackData:
    """
    Class containing fake datasets by us :)
    """

    # Load a DataFrame with a specific version of a CSV
    file_path = "../files/shuffled_df_LLM101_filtered.csv"
    df_o = pd.read_csv(file_path)

    enr_path = "../files/output_df.csv"
    df = pd.read_csv(enr_path)


# print(FeedbackData.df["Feedback"][0])


# Structured output
### Pydantic-objekter

In [11]:
from pydantic import BaseModel, Field
from typing import Literal, Optional


class Categorize_search(BaseModel):
    """Categorization of IT-questionaire feedback."""

    fb: str = Field(description="Return just the feedback verbatim.")
    categories: Literal[
        "Technology",
        "IT support",
        "Network",
        "Other",
    ] = Field(
        description="Categorize the feedback into the most fitting category. If the categories provided are not a perfect fit, default to 'Other'."
    )
    if_other: str = Field(
        description="If you chose to categorize the feedback as 'Other', return an explanation as to why and what category you think would be the best fit for the feedback."
    )


class Categorize_bound(BaseModel):
    # Begrenser kategoriene modellen kan velge mellom. Den får kun lov til å putte feedbacken i en av de forhåndsbestemte kategoriene.
    """Categorization of IT-questionaire feedback."""

    fb: str = Field(description="Return just the feedback verbatim.")
    categories: Literal[
        "Cyber security",
        "IT training",
        "IT support",
        "Quality of technology",
        "Data quality",
        "Network",
    ] = Field(description="Categorize the feedback into the most fitting category.")
    rating: Optional[int] = Field(
        description="Your certainty of the corectness of the best category on a scale from 1-10"
    )
    reason: str = Field(
        description="Give a short scentence as to why you think this category is the best fit."
    )


class Categorize_think(BaseModel):
    """Categorization of IT-questionaire feedback."""

    cot: str = Field(
        description="Think about and explain why this category is the most fitting one and why you chose it."
    )
    categories: Literal[
        "Cyber secyrity", "IT training", "IT support", "Technology quality"
    ] = Field(description="Categorize the feedback into the most fitting category.")


class Categorize(BaseModel):
    "Categorization of feedback on IT-services."

    # Thoughts: Forklar tankegangen din.
    fb: str = Field(description="Return just the feedback verbatim.")
    cat_1: str = Field(description="The best fitting general category")
    rating: Optional[int] = Field(
        description="Your certainty of the corectness of the best category on a scale from 1-10"
    )


class Enrich(BaseModel):
    "Enrichment of dataset to make it more human"

    cat_1: str = Field(
        description="Return the enriched dataset in a comma separated format like the format of the input dataset"
    )


class Sentiment(BaseModel):
    "Provide a sentiment analysis of feedback"

    fb: str = Field(description="Return just the feedback verbatim.")
    sentiment: str = Field(
        description="Return the sentiment analysis of the feedback as a label. Either positive, negative or mixed."
    )
    depth_sent: str = Field(
        description="Return a more nuanced analysis of the sentiment. Is there more complexity to the sentiment than just positive or negative?"
    )

    feeling: str = Field(
        description="What feelings are most predominant in the sentiment of the feedback? Return just the feeling(s)."
    )


# Kategorisering

In [12]:
def categorize_feedback(data, struktur, llm_model):
    # Prompt
    task = f"""
    The following feedback is from an internal survey at 'IT and Things Company' where they asked their employees for feedback on their IT-services in general.
    Catgorize the feedback: {data}.
    """

    # Giving the task to LLM
    structured_llm = llm_model.with_structured_output(
        struktur, method="function_calling"
    )

    response = structured_llm.invoke(task)

    # Return the response
    return response

In [13]:
n = 2

### 1. Kun forhåndsbestemte kategorier

In [14]:
# Bound categorization

responses_3O = {}
responses_4O = {}

for i in range(n):
    responses_3O[str(i)] = categorize_feedback(
        FeedbackData.df["Feedback"][i], Categorize_bound, LlmRes.llm
    )
    responses_4O[str(i)] = categorize_feedback(FeedbackData.df["Feedback"][i], Categorize_bound, Llm.llm)
    print(f"\n-----Feedback-----\n {responses_3O[str(i)].fb}\n")
    print(
        f"-----Category-----\n 4O: {responses_4O[str(i)].categories}\n 3O: {responses_3O[str(i)].categories}\n"
    )
    print(
        f"-----Rating of certainty-----\n 4O: {responses_4O[str(i)].rating}\n 3O: {responses_3O[str(i)].rating}\n"
    )
    print(
        f"-----Reason-----\n 4O: {responses_4O[str(i)].reason}\n 3O: {responses_3O[str(i)].reason}\n\n"
    )



-----Feedback-----
 Although the system is old and its documentation remains thorough, the outdated interface sometimes creates subtle moments of confusion and occasional frustration during peak hours.

-----Category-----
 4O: Quality of technology
 3O: Quality of technology

-----Rating of certainty-----
 4O: 8
 3O: 9

-----Reason-----
 4O: The feedback highlights issues with the outdated interface of the system, which directly relates to the quality of technology.
 3O: The feedback highlights issues related to the system's outdated user interface causing confusion and frustration, which directly points to the overall quality and usability of the technology.



-----Feedback-----
 The step-by-step training guides for warehouse operations are mostly clear and effective; I often feel confident and even joyful when things click, although at times the dry tone leaves me wishing for more engaging details..

-----Category-----
 4O: IT training
 3O: IT training

-----Rating of certainty----

### 2. Modellen velger fritt kategorier.

In [15]:

responses_3O = {}
responses_4O = {}

for i in range(n):
    responses_3O[str(i)] = categorize_feedback(
        FeedbackData.df["Feedback"][i], Categorize ,LlmRes.llm
    )
    responses_4O[str(i)] = categorize_feedback(FeedbackData.df["Feedback"][i], Categorize, Llm.llm)
    print(f"\n-----Feedback-----\n {responses_3O[str(i)].fb}\n")
    print(
        f"-----Category 1-----\n 4O: {responses_4O[str(i)].cat_1}\n 3O: {responses_3O[str(i)].cat_1}\n"
    )
    print(
        f"-----Rating of certainty-----\n 4O: {responses_4O[str(i)].rating}\n 3O: {responses_3O[str(i)].rating}\n\n"
    )



-----Feedback-----
 Although the system is old and its documentation remains thorough, the outdated interface sometimes creates subtle moments of confusion and occasional frustration during peak hours..

-----Category 1-----
 4O: User Experience
 3O: User Experience / Usability

-----Rating of certainty-----
 4O: 8
 3O: 9



-----Feedback-----
 The step-by-step training guides for warehouse operations are mostly clear and effective; I often feel confident and even joyful when things click, although at times the dry tone leaves me wishing for more engaging details..

-----Category 1-----
 4O: Training and Documentation
 3O: Training & Documentation

-----Rating of certainty-----
 4O: 8
 3O: 9




### 3. Gi noen kategorier + 'other' 

In [16]:

responses_3O = {}
responses_4O = {}

for i in range(n):
    responses_3O[i] = categorize_feedback(FeedbackData.df["Feedback"][i], Categorize_search, LlmRes.llm)
    responses_4O[i] = categorize_feedback(FeedbackData.df["Feedback"][i], Categorize_search, Llm.llm)
    print(f"\n-----Feedback-----\n {responses_3O[i].fb}\n")
    print(
        f"-----Category-----\n 4O: {responses_4O[i].categories}\n 3O: {responses_3O[i].categories}\n"
    )
    print(
        f"-----If other-----\n 4O: {responses_4O[i].if_other}\n 3O: {responses_3O[i].if_other}\n"
    )



-----Feedback-----
 Although the system is old and its documentation remains thorough, the outdated interface sometimes creates subtle moments of confusion and occasional frustration during peak hours.

-----Category-----
 4O: Technology
 3O: Technology

-----If other-----
 4O: 
 3O: 


-----Feedback-----
 The step-by-step training guides for warehouse operations are mostly clear and effective; I often feel confident and even joyful when things click, although at times the dry tone leaves me wishing for more engaging details.

-----Category-----
 4O: Other
 3O: IT support

-----If other-----
 4O: This feedback pertains to training materials rather than a specific IT service, making it difficult to categorize under Technology, IT support, or Network. The best fit would be 'Training and Development'.
 3O: 

